In [94]:
# Clone the CodeXGLUE repository
!git clone https://github.com/microsoft/CodeXGLUE.git

# Navigate to the CodeBLEU directory
%cd CodeXGLUE/Code-Code/CodeBLEU


fatal: destination path 'CodeXGLUE' already exists and is not an empty directory.
[Errno 2] No such file or directory: 'CodeXGLUE/Code-Code/CodeBLEU'
/home/vishal/Projects/llm_for_code_review/CORE_llm


/home/vishal/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [95]:
with open('go_code_in.go', 'r') as f_in:
    input_code = f_in.read()

with open('go_code_out.go', 'r') as f_out:
    candidate_code = f_out.read()

with open('go_code_ref.go', 'r') as f_ref:
    reference_code = f_ref.read()

In [96]:
import sys
sys.path.append('/content/CodeXGLUE/Code-Code/CodeBLEU')
from codebleu import bleu

# Your provided code snippets
reference_texts = [reference_code]

candidate_texts = [candidate_code]

# Calculate CodeBLEU score
def calculate_codebleu(reference_texts, candidate_texts):
    weights = [0.25, 0.25, 0.25, 0.25]
    lang = 'go'

    scores = []
    for ref, cand in zip(reference_texts, candidate_texts):
        code_bleu_score = bleu.corpus_bleu([[ref]], [cand], weights)
        scores.append(code_bleu_score)

    return scores

# Calculate CodeBLEU scores
codebleu_scores = calculate_codebleu(reference_texts, candidate_texts)

# Print CodeBLEU scores
for i, score in enumerate(codebleu_scores):
    print(f"CodeBLEU score for candidate {i+1}: {score:.4f}")


CodeBLEU score for candidate 1: 0.8086


In [97]:
import ast
import astor
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import Levenshtein

nltk.download('punkt')
nltk.download('stopwords')

def tokenize_code(code):
    tokens = word_tokenize(code)
    tokens = [token for token in tokens if token.isalnum()]
    return tokens

def cosine_similarity_score(code1, code2):
    vectorizer = TfidfVectorizer().fit_transform([code1, code2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

def jaccard_similarity_score(code1, code2):
    tokens1 = set(tokenize_code(code1))
    tokens2 = set(tokenize_code(code2))
    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    return len(intersection) / len(union)

def levenshtein_distance_score(code1, code2):
    return Levenshtein.ratio(code1, code2)

def tree_edit_distance_score(code1, code2):
    tree1 = ast.parse(code1)
    tree2 = ast.parse(code2)
    return compute_tree_edit_distance(tree1, tree2)

def compute_tree_edit_distance(tree1, tree2):
    # Simplified tree edit distance calculation
    def node_label(node):
        return type(node).__name__

    def children(node):
        return [child for child in ast.iter_child_nodes(node)]

    def simple_tree_edit_distance(tree1, tree2):
        label1 = node_label(tree1)
        label2 = node_label(tree2)
        if label1 != label2:
            return 1 + max(len(children(tree1)), len(children(tree2)))
        return sum(simple_tree_edit_distance(c1, c2) for c1, c2 in zip(children(tree1), children(tree2)))

    return simple_tree_edit_distance(tree1, tree2)


# Calculate and print scores
print("Cosine Similarity Score:", cosine_similarity_score(reference_code, candidate_code))
print("Jaccard Similarity Score:", jaccard_similarity_score(reference_code, candidate_code))
print("Levenshtein Distance Score:", levenshtein_distance_score(reference_code, candidate_code))


Cosine Similarity Score: 0.9072751917045446
Jaccard Similarity Score: 0.7777777777777778
Levenshtein Distance Score: 0.812235294117647


[nltk_data] Downloading package punkt to /home/vishal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vishal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [98]:
import ast
import astor
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import Levenshtein

nltk.download('punkt')
nltk.download('stopwords')

def tokenize_code(code):
    tokens = word_tokenize(code)
    tokens = [token for token in tokens if token.isalnum()]
    return tokens

def cosine_similarity_score(code1, code2):
    vectorizer = TfidfVectorizer().fit_transform([code1, code2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

def jaccard_similarity_score(code1, code2):
    tokens1 = set(tokenize_code(code1))
    tokens2 = set(tokenize_code(code2))
    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    return len(intersection) / len(union)

def levenshtein_distance_score(code1, code2):
    return Levenshtein.ratio(code1, code2)

def clean_code(code):
    # Remove comments and ensure no unterminated strings
    return '\n'.join([line for line in code.split('\n') if not line.strip().startswith('//') and not line.strip().startswith('#')])

def tree_edit_distance_score(code1, code2):
    try:
        tree1 = ast.parse(clean_code(code1))
        tree2 = ast.parse(clean_code(code2))
    except SyntaxError:
        return float('inf')  # Return an infinite distance if parsing fails

    def simple_tree_edit_distance(tree1, tree2):
        label1 = type(tree1).__name__
        label2 = type(tree2).__name__
        if label1 != label2:
            return 1 + max(len(list(ast.iter_child_nodes(tree1))), len(list(ast.iter_child_nodes(tree2))))
        return sum(simple_tree_edit_distance(c1, c2) for c1, c2 in zip(list(ast.iter_child_nodes(tree1)), list(ast.iter_child_nodes(tree2))))

    return simple_tree_edit_distance(tree1, tree2)


# Clean the code (remove comments and ensure no unterminated strings)
cleaned_reference = clean_code(reference_code)
cleaned_candidate = clean_code(candidate_code)

# Calculate and print scores
print("Cosine Similarity Score:", cosine_similarity_score(cleaned_reference, cleaned_candidate))
print("Jaccard Similarity Score:", jaccard_similarity_score(cleaned_reference, cleaned_candidate))
print("Levenshtein Distance Score:", levenshtein_distance_score(cleaned_reference, cleaned_candidate))

Cosine Similarity Score: 0.8879618576461086
Jaccard Similarity Score: 0.85
Levenshtein Distance Score: 0.834059022738268


[nltk_data] Downloading package punkt to /home/vishal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vishal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
from difflib import SequenceMatcher

# Function to calculate structural similarity using SequenceMatcher
def structural_similarity(code1, code2):
    return SequenceMatcher(None, code1, code2).ratio()

# Calculate structural similarity between the reference and candidate Go code
similarity_score = structural_similarity(reference_code, candidate_code)
print("Structural Similarity Score:", similarity_score)


Structural Similarity Score: 0.5468235294117647
